In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os

# Path to your Drive
drive_root = '/content/drive/MyDrive'

# Project folder inside Drive
project_folder = os.path.join(drive_root, 'ImageForgeryDetection')

# Dataset already uploaded
dataset_path = os.path.join(drive_root, 'data_set')  # your dataset should have 'real_images' and 'fake_images'

# Create project folder if it doesn't exist
os.makedirs(project_folder, exist_ok=True)
print(f"Project folder set at: {project_folder}")


Project folder set at: /content/drive/MyDrive/ImageForgeryDetection


In [ ]:
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam


In [ ]:
image_size = (600, 600)
batch_size = 8  # Smaller batch to avoid memory crash

datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_data = datagen.flow_from_directory(
    dataset_path,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='binary',
    subset='training'
)

val_data = datagen.flow_from_directory(
    dataset_path,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='binary',
    subset='validation'
)


Found 1648 images belonging to 2 classes.
Found 412 images belonging to 2 classes.


In [ ]:
model = Sequential([
    Conv2D(16, (3,3), activation='relu', input_shape=(600, 600, 3)),
    MaxPooling2D(2,2),

    Conv2D(32, (3,3), activation='relu'),
    MaxPooling2D(2,2),

    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),

    Flatten(),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])


In [ ]:
model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])

epochs = 10
model.fit(train_data, validation_data=val_data, epochs=epochs)


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
206/206 ━━━━━━━━━━━━━━━━━━━━ 472s 2s/step - accuracy: 0.5157 - loss: 0.8847 - val_accuracy: 0.5243 - val_loss: 0.6927
Epoch 2/10
206/206 ━━━━━━━━━━━━━━━━━━━━ 42s 204ms/step - accuracy: 0.5234 - loss: 0.6949 - val_accuracy: 0.5316 - val_loss: 0.6922
Epoch 3/10
206/206 ━━━━━━━━━━━━━━━━━━━━ 45s 221ms/step - accuracy: 0.5341 - loss: 0.6876 - val_accuracy: 0.5413 - val_loss: 0.7277
Epoch 4/10
206/206 ━━━━━━━━━━━━━━━━━━━━ 42s 202ms/step - accuracy: 0.5977 - loss: 0.6686 - val_accuracy: 0.5316 - val_loss: 0.7039
Epoch 5/10
206/206 ━━━━━━━━━━━━━━━━━━━━ 41s 201ms/step - accuracy: 0.6982 - loss: 0.5948 - val_accuracy: 0.5364 - val_loss: 0.7451
Epoch 6/10
206/206 ━━━━━━━━━━━━━━━━━━━━ 44s 213ms/step - accuracy: 0.7813 - loss: 0.4668 - val_accuracy: 0.5437 - val_loss: 0.9295
Epoch 7/10
206/206 ━━━━━━━━━━━━━━━━━━━━ 41s 201ms/step - accuracy: 0.8743 - loss: 0.3019 - val_accuracy: 0.5073 - val_loss: 1.2975
Epoch 8/10
206/206 ━━━━━━━━━━━━━━━━━━━━ 42s 204ms/step - accuracy: 0.9209 - loss: 0.1

In [ ]:
model_save_path = os.path.join(project_folder, 'image_classifier_600x600.h5')
model.save(model_save_path)
print("Model saved at:", model_save_path)


Model saved at: /content/drive/MyDrive/ImageForgeryDetection/image_classifier_600x600.h5


In [ ]:
!pip install gradio


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.9/46.9 MB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 29.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 137.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 6.6 MB/s eta 0:00:00


In [ ]:
from tensorflow.keras.models import load_model
import numpy as np
from tensorflow.keras.preprocessing import image
import gradio as gr
import os

# Load your model from Drive
model_path = '/content/drive/MyDrive/ImageForgeryDetection/image_classifier_600x600.h5'
model = load_model(model_path)


In [ ]:
def predict_image(img):
    img = img.resize((600, 600))  # Resize to match training size
    img_array = image.img_to_array(img)
    img_array = img_array / 255.0  # Normalize
    img_array = np.expand_dims(img_array, axis=0)

    prediction = model.predict(img_array)[0][0]

    if prediction < 0.5:
        return "FAKE IMAGE ❌"
    else:
        return "REAL IMAGE ✅"


In [ ]:
interface = gr.Interface(
    fn=predict_image,
    inputs=gr.Image(type="pil"),
    outputs="text",
    title="Image Forgery Detection",
    description="Upload an image to check if it's real or tampered."
)

interface.launch()


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://dca867c9143935c623.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
